In [201]:
import random
import numpy as np
import pandas as pd
from gensim.models import word2vec

In [202]:
question_df = pd.read_csv('../data/input/questions.csv')
question_df = question_df.dropna().reset_index(drop=True)
question_df.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [203]:
question_tags = question_df['tags'].values
question_tags_list = [str(t).split(' ') for t in question_tags]
shuffled_tags_list = []
for t in question_tags_list:
    random.shuffle(t)
    shuffled_tags_list.append(t)

question_tags_list = question_tags_list + shuffled_tags_list

In [238]:
def category2vec_with_w2v(df, group_name, category_name, w2v_params):
    df_ = df[[group_name, category_name]].groupby(group_name).agg(list)
    df_[category_name] = df_[category_name].apply(lambda x: x[0].split(' ')).tolist()
    
    category_list = df_[category_name].tolist()
    shuffled_category_list = []
    for c in category_list:
        random.shuffle(c)
        shuffled_tags_list.append(c)
        
    category_list += shuffled_category_list

    n_components = w2v_params['size']
    model = word2vec.Word2Vec(category_list, **w2v_params)
    vocab_keys = list(model.wv.vocab.keys())
    w2v_array = np.zeros((len(vocab_keys), n_components))
    
    for i, v in enumerate(vocab_keys):
        w2v_array[i, :] = model.wv[v]
    vocab_vectors_df = pd.DataFrame(w2v_array, columns=[f'v{i}' for i in range(n_components)])
    vocab_vectors_df[category_name] = vocab_keys
    
#     df = pd.merge()
    
    return vocab_vectors_df

In [240]:
w2v_params = {
    "size": n_components,
    "iter": 10,
    "window": 100,
    "seed": 2020,
    "min_count": 1,
    "workers": -1
}

question_df = pd.read_csv('../data/input/questions.csv')
question_df = question_df.dropna().reset_index(drop=True)
question_df

# question_exploded_df = question_df[['question_id', 'tags_list']].explode('tags_list').reset_index(drop=True)

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38
...,...,...,...,...,...
13517,13518,13518,3,5,14
13518,13519,13519,3,5,8
13519,13520,13520,2,5,73
13520,13521,13521,0,5,125


In [ ]:
category2vec_with_w2v(question_exploded_df, group_name='question_id', category_name='tags', w2v_params=w2v_params)

In [190]:
n_components = 16

w2v_params = {
    "size": n_components,
    "iter": 10,
    "window": 100,
    "seed": 2020,
    "min_count": 1,
    "workers": -1
}

model = word2vec.Word2Vec(question_tags_list, **w2v_params)
vocab_keys = list(model.wv.vocab.keys())     
w2v_array = np.zeros((len(vocab_keys), n_components))

for i, v in enumerate(vocab_keys):
    w2v_array[i, :] = model.wv[v]
vocab_vectors_df = pd.DataFrame(w2v_array, columns=[f'v{i}' for i in range(n_components)])
vocab_vectors_df['tag'] = vocab_keys

question_df['tags_list'] = question_df['tags'].apply(lambda x: str(x).split(' '))
question_exploded_df = question_df[['question_id', 'tags_list']].explode('tags_list').reset_index(drop=True)
question_exploded_df.rename(columns={'tags_list': 'tag'}, inplace=True)

for i in range(n_components):
    le = dict(vocab_vectors_df[['tag', f'v{i}']].values)
    question_exploded_df[f'v{i}'] = question_exploded_df['tag'].map(le)
    
question_exploded_df.groupby('question_id')['v1'].agg({'mean', 'median', 'std', 'max', 'min'}).add_prefix('v1_')

,v1_max,v1_std,v1_mean,v1_median,v1_min
question_id,,,,,
0,0.028278,0.014883,0.013295,0.014129,-0.003358
1,0.023108,0.013119,0.014709,0.021427,-0.000408
2,0.023108,0.016220,-0.000450,-0.005809,-0.013288
3,0.023108,0.012919,0.012506,0.015137,-0.003358
4,0.023108,0.015583,0.002797,0.000896,-0.013711
...,...,...,...,...,...
13518,0.026483,NaN,0.026483,0.026483,0.026483
13519,-0.021106,NaN,-0.021106,-0.021106,-0.021106
13520,0.011713,NaN,0.011713,0.011713,0.011713


In [193]:
question_exploded_df[['question_id', 'tag']].groupby('question_id').agg(list)

,tag
question_id,
0,"[51, 131, 162, 38]"
1,"[131, 36, 81]"
2,"[131, 101, 162, 92]"
3,"[131, 149, 162, 29]"
4,"[131, 5, 162, 38]"
...,...
13518,[14]
13519,[8]
13520,[73]
